<a href="https://colab.research.google.com/github/anitayadav3/EmotionRecognitionInConversation/blob/master/Practice_Workspace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/My Drive/DialogueRNN

/content/gdrive/My Drive/DialogueRNN


In [ ]:
# !python train_IEMOCAP.py --dropout 0.4 --lr 0.0003 --batch-size 32 --class-weight --l2 0.0

**Train_IEMOCAP file opened for practise**

In [3]:
import numpy as np
np.random.seed(1234)
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torch.optim as optim
import argparse
import time
import pickle
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score,\
                        classification_report, precision_recall_fscore_support
from model import BiModel, Model, MaskedNLLLoss
from dataloader import IEMOCAPDataset

In [4]:
def get_train_valid_sampler(trainset, valid=0.1):
    size = len(trainset)
    idx = list(range(size))
    split = int(valid*size)
    return SubsetRandomSampler(idx[split:]), SubsetRandomSampler(idx[:split])

def get_IEMOCAP_loaders(path, batch_size=32, valid=0.1, num_workers=0, pin_memory=False):
    trainset = IEMOCAPDataset(path=path)
    train_sampler, valid_sampler = get_train_valid_sampler(trainset, valid)
    train_loader = DataLoader(trainset,
                              batch_size=batch_size,
                              sampler=train_sampler,
                              collate_fn=trainset.collate_fn,
                              num_workers=num_workers,
                              pin_memory=pin_memory)
    valid_loader = DataLoader(trainset,
                              batch_size=batch_size,
                              sampler=valid_sampler,
                              collate_fn=trainset.collate_fn,
                              num_workers=num_workers,
                              pin_memory=pin_memory)

    testset = IEMOCAPDataset(path=path, train=False)
    test_loader = DataLoader(testset,
                             batch_size=batch_size,
                             collate_fn=testset.collate_fn,
                             num_workers=num_workers,
                             pin_memory=pin_memory)

    return train_loader, valid_loader, test_loader

def train_or_eval_model(model, loss_function, dataloader, epoch, optimizer=None, train=False):
    losses = []
    preds = []
    labels = []
    masks = []
    alphas, alphas_f, alphas_b, vids = [], [], [], []
    assert not train or optimizer!=None
    if train:
        model.train()
    else:
        model.eval()
    for data in dataloader:
        if train:
            optimizer.zero_grad()
        # import ipdb;ipdb.set_trace()
        textf, visuf, acouf, qmask, umask, label =\
                [d.cuda() for d in data[:-1]] if cuda else data[:-1]
        #log_prob = model(torch.cat((textf,acouf,visuf),dim=-1), qmask,umask,att2=True) # seq_len, batch, n_classes
        log_prob, alpha, alpha_f, alpha_b = model(textf, qmask,umask,att2=True) # seq_len, batch, n_classes
        lp_ = log_prob.transpose(0,1).contiguous().view(-1,log_prob.size()[2]) # batch*seq_len, n_classes
        labels_ = label.view(-1) # batch*seq_len
        loss = loss_function(lp_, labels_, umask)

        pred_ = torch.argmax(lp_,1) # batch*seq_len
        preds.append(pred_.data.cpu().numpy())
        labels.append(labels_.data.cpu().numpy())
        masks.append(umask.view(-1).cpu().numpy())

        losses.append(loss.item()*masks[-1].sum())
        if train:
            loss.backward()
            if args.tensorboard:
                for param in model.named_parameters():
                    writer.add_histogram(param[0], param[1].grad, epoch)
            optimizer.step()
        else:
            alphas += alpha
            alphas_f += alpha_f
            alphas_b += alpha_b
            vids += data[-1]

    if preds!=[]:
        preds  = np.concatenate(preds)
        labels = np.concatenate(labels)
        masks  = np.concatenate(masks)
    else:
        return float('nan'), float('nan'), [], [], [], float('nan'),[]

    avg_loss = round(np.sum(losses)/np.sum(masks),4)
    avg_accuracy = round(accuracy_score(labels,preds,sample_weight=masks)*100,2)
    avg_fscore = round(f1_score(labels,preds,sample_weight=masks,average='weighted')*100,2)
    return avg_loss, avg_accuracy, labels, preds, masks,avg_fscore, [alphas, alphas_f, alphas_b, vids]

In [5]:
batch_size = 32
n_classes  = 6
cuda = torch.cuda.is_available()
n_epochs   = 60

In [6]:
D_m = 100
D_g = 500
D_p = 500
D_e = 300
D_h = 300
D_a = 100 # concat attention

In [7]:
model = BiModel(D_m, D_g, D_p, D_e, D_h,
                n_classes=n_classes,
                listener_state=False,
                context_attention='general',
                dropout_rec=0.1,
                dropout=0.4)

In [8]:
model.cuda()

BiModel(
  (dropout): Dropout(p=0.4, inplace=False)
  (dropout_rec): Dropout(p=0.55, inplace=False)
  (dialog_rnn_f): DialogueRNN(
    (dropout): Dropout(p=0.1, inplace=False)
    (dialogue_cell): DialogueRNNCell(
      (g_cell): GRUCell(600, 500)
      (p_cell): GRUCell(600, 500)
      (e_cell): GRUCell(500, 300)
      (dropout): Dropout(p=0.1, inplace=False)
      (attention): MatchingAttention(
        (transform): Linear(in_features=100, out_features=500, bias=False)
      )
    )
  )
  (dialog_rnn_r): DialogueRNN(
    (dropout): Dropout(p=0.1, inplace=False)
    (dialogue_cell): DialogueRNNCell(
      (g_cell): GRUCell(600, 500)
      (p_cell): GRUCell(600, 500)
      (e_cell): GRUCell(500, 300)
      (dropout): Dropout(p=0.1, inplace=False)
      (attention): MatchingAttention(
        (transform): Linear(in_features=100, out_features=500, bias=False)
      )
    )
  )
  (linear): Linear(in_features=600, out_features=600, bias=True)
  (smax_fc): Linear(in_features=600, out_featur

In [9]:
loss_weights = torch.FloatTensor([
                                    1/0.086747,
                                    1/0.144406,
                                    1/0.227883,
                                    1/0.160585,
                                    1/0.127711,
                                    1/0.252668,
                                      ])

In [10]:
loss_function = MaskedNLLLoss(loss_weights.cuda())

In [11]:
optimizer = optim.Adam(model.parameters(),
                           lr=0.0003,
                           weight_decay=0.0)

In [12]:
train_loader, valid_loader, test_loader =\
            get_IEMOCAP_loaders('./IEMOCAP_features/IEMOCAP_features_raw.pkl',
                                valid=0.0,
                                batch_size=batch_size,
                                num_workers=2)

In [13]:
best_loss, best_label, best_pred, best_mask = None, None, None, None

**nn.GRUCell : (input_size, hidden_size, bias (bool, default: True) )**

In [ ]:
test_cell = nn.GRUCell(100,100,True)

Assigned variables from functions :

Loss Function : Negative Log Likelihood

**train_or_eval_model FUNCTION Opened HERE**

In [14]:
losses = []
preds = []
labels = []
masks = []
alphas, alphas_f, alphas_b, vids = [], [], [], []

In [17]:
dataloader=train_loader

In [22]:
for data in dataloader:
  optimizer.zero_grad()
  textf, visuf, acouf, qmask, umask, label =\
                [d.cuda() for d in data[:-1]] if cuda else data[:-1]
  log_prob, alpha, alpha_f, alpha_b = model(textf, qmask,umask,att2=True)
  lp_ = log_prob.transpose(0,1).contiguous().view(-1,log_prob.size()[2])
  labels_ = label.view(-1)
  loss = loss_function(lp_, labels_, umask)
  pred_ = torch.argmax(lp_,1)
  preds.append(pred_.data.cpu().numpy())
  labels.append(labels_.data.cpu().numpy())
  masks.append(umask.view(-1).cpu().numpy())
  losses.append(loss.item()*masks[-1].sum())
  loss.backward()
  optimizer.step()

In [24]:
if preds!=[]:
  preds  = np.concatenate(preds)
  labels = np.concatenate(labels)
  masks  = np.concatenate(masks)
else:
  print(float('nan'), float('nan'), [], [], [], float('nan'),[])

In [25]:
avg_loss = round(np.sum(losses)/np.sum(masks),4)
avg_accuracy = round(accuracy_score(labels,preds,sample_weight=masks)*100,2)
avg_fscore = round(f1_score(labels,preds,sample_weight=masks,average='weighted')*100,2)

In [26]:
print(avg_loss)
print(avg_accuracy)
print(labels)
print(preds)
print(masks)
print(avg_fscore)
print([alphas, alphas_f, alphas_b, vids])

1.6275
40.34
[4 4 4 ... 0 0 0]
[2 1 2 ... 0 4 4]
[1. 1. 1. ... 0. 0. 0.]
38.64
[[], [], [], []]


**Do not run below this**

In [ ]:
for e in range(n_epochs):
  start_time = time.time()
  train_loss, train_acc, _,_,_,train_fscore,_= train_or_eval_model(model, loss_function,train_loader, e, optimizer, True)
  valid_loss, valid_acc, _,_,_,val_fscore,_= train_or_eval_model(model, loss_function, valid_loader, e)
  test_loss, test_acc, test_label, test_pred, test_mask, test_fscore, attentions = train_or_eval_model(model, loss_function, test_loader, e)
  if best_loss == None or best_loss > test_loss:
    best_loss, best_label, best_pred, best_mask, best_attn =\
                    test_loss, test_label, test_pred, test_mask, attentions

RuntimeError: ignored

In [ ]:
print('Test performance..')
print('Loss {} accuracy {}'.format(best_loss,
                                  round(accuracy_score(best_label,best_pred,sample_weight=best_mask)*100,2)))
print(classification_report(best_label,best_pred,sample_weight=best_mask,digits=4))
print(confusion_matrix(best_label,best_pred,sample_weight=best_mask))

Test performance..


ValueError: ignored